In [39]:
from collections import defaultdict
import sys

In [40]:
DOWN = 1
RIGHT = 2
DIAG = 3

In [50]:
def lcs_backtrack(v, w):    
    n = len(v)
    m = len(w)
    
    backtrack = []
    for i in range(n):
        backtrack.append([0]*m)    
    
    grid = []
    for i in range(n+1):
        grid.append([0]*(m+1))
        
    for i in range(1,n+1):
        for j in range(1,m+1):
            match = 0
            if v[i-1] == w[j-1]:
                match = 1
            grid[i][j] = max(
                grid[i-1][j],
                grid[i][j-1],
                grid[i-1][j-1] + match
            )
            if grid[i][j] == grid[i-1][j]:
                backtrack[i-1][j-1] = DOWN
            elif grid[i][j] == grid[i][j-1]:
                backtrack[i-1][j-1] = RIGHT
            elif grid[i][j] == grid[i-1][j-1] + match:
                backtrack[i-1][j-1] = DIAG
                
    return grid, backtrack

In [51]:
v="AACCTTGG"
w="ACACTGTGA"

In [52]:
grid, backtrack = lcs_backtrack(v, w)
backtrack

[[3, 2, 2, 2, 2, 2, 2, 2, 2],
 [1, 1, 3, 2, 2, 2, 2, 2, 2],
 [1, 3, 1, 3, 2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 3, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 1, 3, 2, 2],
 [1, 1, 1, 1, 1, 3, 1, 3, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [62]:
def output_lcs(backtrack, v, i, j):
    if i == 0 or j == 0:
        return ""
    if backtrack[i-1][j-1] == DOWN:
        return output_lcs(backtrack, v, i-1, j)
    elif backtrack[i-1][j-1] == RIGHT:
        return output_lcs(backtrack, v, i, j-1)
    else:
        return output_lcs(backtrack, v, i-1, j-1) + v[i-1]

In [63]:
output_lcs(backtrack, v, len(v), len(w))

'ACTTAATCCAGTACCGCTATGCAGGCGGCAGATATCCAACGCAGTAAACCCTAGGTTCGCTTTTCCACTTACAACGCTTCCGCGTGAATTCCAAGCTCAAACCAGACTGCTACGCAGCTCGTCAGATGCTCGGCGACACAGGGATTATGAAATAGTAGAGGGTAAATTCGTCCATGAACCGTCTTCGTGTATCCCACTACTTATCCCTGACGGAACTCTGAGATGTTGGGCACGCTTTCAGCCGTATATACAGCTTTCCAGCCATTTCCGTCGCCAGAAGAGTCCACCGTATAACAAGGCGACCTATCGAACTCGAGGAGTACGAACGCCGCGATCACGAAATATTGTAAAGTGACGGTTTGGGAGGTGGGAACATACGAATGAGATTCATGGCTTCCATAGAAGCTGTGACACAACAATTTCGCAAACGGATCGGACAACAAACCTCCCCCGTACTGTGTTCCCAATACACAGCTCTGTCAGGTCAATTATTAACTCCAGTATTAGTTTCCCCATCTAGCATCGGTAAAGCACATGTTTTGAACTCGCCAACGCAGGTACAGCCACGTAATTATATTACGTGCAGTGATCCTAAGCGAGGGCTTTACTTTCTTT'

In [64]:
with open("../data/dataset_245_5.txt", "r") as fin:
    v, w = fin.read().strip().split("\n")
    grid, backtrack = lcs_backtrack(v, w)
    print(output_lcs(backtrack, v, len(v), len(w)))

ACTTAATCCAGTACCGCTATGCAGGCGGCAGATATCCAACGCAGTAAACCCTAGGTTCGCTTTTCCACTTACAACGCTTCCGCGTGAATTCCAAGCTCAAACCAGACTGCTACGCAGCTCGTCAGATGCTCGGCGACACAGGGATTATGAAATAGTAGAGGGTAAATTCGTCCATGAACCGTCTTCGTGTATCCCACTACTTATCCCTGACGGAACTCTGAGATGTTGGGCACGCTTTCAGCCGTATATACAGCTTTCCAGCCATTTCCGTCGCCAGAAGAGTCCACCGTATAACAAGGCGACCTATCGAACTCGAGGAGTACGAACGCCGCGATCACGAAATATTGTAAAGTGACGGTTTGGGAGGTGGGAACATACGAATGAGATTCATGGCTTCCATAGAAGCTGTGACACAACAATTTCGCAAACGGATCGGACAACAAACCTCCCCCGTACTGTGTTCCCAATACACAGCTCTGTCAGGTCAATTATTAACTCCAGTATTAGTTTCCCCATCTAGCATCGGTAAAGCACATGTTTTGAACTCGCCAACGCAGGTACAGCCACGTAATTATATTACGTGCAGTGATCCTAAGCGAGGGCTTTACTTTCTTT


In [12]:
input = """0
4
0->1:7
0->2:4
2->3:2
1->4:1
3->4:3
"""

In [82]:
def process_input(input):
    lines = input.split("\n")[:-1]
    src = int(lines[0])
    snk = int(lines[1])
    weights = dict()
    predecessors = defaultdict(list)
    for line in lines[2:]:
        src_node, dst = line.split("->")
        dst_node, weight = dst.split(":")
        src_node = int(src_node)
        dst_node = int(dst_node)
        weight = int(weight)
        weights[(src_node, dst_node)] = weight
        predecessors[dst_node].append(src_node)
    return src, snk, weights, predecessors

In [14]:
src, snk, weights, predecessors = process_input(input)

In [15]:
print(weights, predecessors)

{(0, 1): 7, (0, 2): 4, (2, 3): 2, (1, 4): 1, (3, 4): 3} defaultdict(<class 'list'>, {1: [0], 2: [0], 3: [2], 4: [1, 3]})


In [73]:
def longest_path(src, snk, weights, predecessors):
    backtrack = defaultdict(list)
    scores = [-sys.maxsize]*(snk-src+1)
    scores[src] = 0
    for node in range(src+1, snk+1):
        node_scores = []
        node_predecessors = predecessors.get(node,[])
        if node_predecessors == []:
            continue
        for predecessor in node_predecessors:
            score = scores[predecessor] + weights.get((predecessor, node))
            node_scores.append(score)
        scores[node] = max(node_scores)
        index = node_scores.index(scores[node])
        backtrack[node].append(node_predecessors[index])
    return scores[snk], backtrack

In [74]:
pathlength, backtrack = longest_path(*process_input(input))

In [75]:
def traceback_path(backtrack, src, snk):
    path = []
    node = snk
    while node in backtrack:
        path.append(node)
        node = backtrack[node][0]
    path.append(node)
    return path

In [19]:
path = traceback_path(backtrack, 0, 4)

In [79]:
def print_result(pathlength, path):
    print(pathlength)
    print("->".join([str(i) for i in path[::-1]]))

In [21]:
print_result(pathlength, path)

9
0->2->3->4


In [23]:
with open("../data/dataset_245_7.txt","r") as fin:
    input = fin.read()
    src, snk, weights, predecessors = process_input(input)
    pathlength, backtrack = longest_path(src, snk, weights, predecessors)
    path = traceback_path(backtrack, src, snk)
    print_result(pathlength, path)

174
0->1->2->3->8->9->10->16->18->26->38->40->44->49


In [65]:
#quize

In [66]:
v="AGACTG"
w="GTACGA"
grid, backtrack = lcs_backtrack(v, w)
output_lcs(backtrack, v, len(v), len(w))

'GACG'

In [88]:
input="""0
6
0->1:5
0->2:6
0->3:5
1->2:2
1->5:4
2->4:4
2->5:3
2->6:5
3->4:7
3->5:5
4->6:2
5->6:1
"""

In [89]:
src, snk, weights, predecessors = process_input(input)

In [90]:
pathlength, backtrack = longest_path(src, snk, weights, predecessors)

In [91]:
path = traceback_path(backtrack, src, snk)

In [92]:
print_result(pathlength, path)

14
0->3->4->6


In [81]:
path

[7]